# Recipe Suggestion Using OpenAI API

It works using OpenAI's gpt-3.5-turbo model.

The user's request and the recipes filtered from the dataset are put into the prompt. At the same time, what kind of output we want and what we pay attention to are explained in this prompt. Accordingly, the model suggests a recipe.

In [2]:
import pandas as pd
from openai import OpenAI
import re
import ast
from typing import List, Tuple, Dict, Any

class RecipeGenerator:
    def __init__(self, api_key: str, csv_file: str):
        # Creating an OpenAI object
        self.client = OpenAI(api_key=api_key)
        # Reading and preprocessing data
        self.df = self.load_preprocess_recipes(csv_file)

    def convert_keywords(self, keywords_str: str):
        """
        Converts the given variable's format from string to list.

        Params:
        --------
        keyword_str (str): variable whose format will be changed

        Return:
        ---------
        keywords_str (list): variable whose format has changed
        """
        # Eğer giriş zaten bir string değilse, olduğu gibi döndür
        if isinstance(keywords_str, str):
            try:
                return ast.literal_eval(keywords_str)
            except:
                return [kw.strip() for kw in keywords_str.strip('"').split('", "')]
        return keywords_str

    def load_preprocess_recipes(self, path: str):
        """
        Loads the data containing the recipes and performs preprocessing operations

        Params:
        --------
        path (str): file location of the data

        Return:
        ---------
        df (DataFrame): Edited recipe data
        """
        # Read data
        df = pd.read_csv(path)
        # Select columns
        chosen_columns = ['Name', 'RecipeCategory', 'Keywords', 'RecipeIngredientParts', 'Calories', 'RecipeInstructions']
        # Fill na values
        df = df[chosen_columns].fillna('')
        # The calorie variable is converted to numeric and non-numeric values ​​are filled with NaN
        df['Calories'] = pd.to_numeric(df['Calories'], errors='coerce')

        # Unwanted characters in variables are removed
        for col in ['Keywords', 'RecipeIngredientParts', 'RecipeInstructions']:
            df[col] = df[col].str.replace("c(", "").str.replace(")", "").apply(self.convert_keywords)

        return df

    def parse_user_input(self, user_input: str):
        """
        Extracts calories, categories, and keywords from user input

        Params:
        ---------
        user_input (str): input that the user wants the recipe for

        Return:
        ---------
        min_calories (int): minimum desired calories
        max_calories (int): maximum desired calories
        category (str): desired category
        keywords (list): searched keywords
        """
        # Check calorie range
        calorie_regex = re.search(r'(\d+)-(\d+)\s*Calories', user_input, re.IGNORECASE)
        if calorie_regex:
            min_calories, max_calories = map(int, calorie_regex.groups())
        else:
            # Checks if there is only one calorie value
            calorie_regex = re.search(r'(\d+)\s*Calories', user_input, re.IGNORECASE)

            min_calories, max_calories = (0, int(calorie_regex.group(1))) if calorie_regex else (0, 1000)

        # Calorie information is removed from the input and the text is cleared
        keywords = re.sub(r'\d+\s*-\s*\d+\s*Calories|\d+\s*Calories', '', user_input, flags=re.IGNORECASE).strip()

        # remaining words are separated by spaces in the list
        words_list = keywords.split()

        # Takes the first word as the category value
        category = words_list[0] if words_list else ""

        # Takes other words as keywords
        keywords = words_list[1:] if len(words_list) > 1 else []

        return min_calories, max_calories, category, keywords

    def find_recipes(self, min_calories: int, max_calories: int, category: str, keywords: List[str]):
        """
        Filters data according to given parameters and finds suitable recipes

        Params:
        ---------
        min_calories (int): desired minimum calories
        max_calories (int): desired maximum calories
        category (str): desired category
        keywords (list): searched keywords

        Return:
        ---------
        filtered_df(DataFrame): filtered data according to desired values
        """

        # Data is filtered by calories
        filtered_df = self.df[(self.df['Calories'] >= min_calories) & (self.df['Calories'] <= max_calories)]
        
        # If there is a category, it is filtered by category
        if category:
            filtered_df = filtered_df[filtered_df['RecipeCategory'].str.contains(category, case=False)]
        
        # If there is a keyword, it is filtered accordingly.
        if keywords:
            for keyword in keywords:
                keyword_lower = keyword.lower()
                filtered_df = filtered_df[
                    filtered_df['Name'].str.lower().str.contains(keyword_lower) |
                    filtered_df['Keywords'].apply(lambda kw_list: keyword_lower in [k.lower() for k in kw_list])]
        
        return filtered_df

    def get_recipe_suggestions(self, user_input: str, num_options: int = 5):
        """
        Returns recipes formatted according to user input

        Params:
        ---------
        user_input (str): Input for which similar recipe is searched
        n (int): Number of recipes to return

        Return:
        ----------
        response (str): All found recipes are returned together
        """

        # Information is taken from the user input
        min_calories, max_calories, category, keywords = self.parse_user_input(user_input)

        # Recipe is found according to the information
        recipes = self.find_recipes(min_calories, max_calories, category, keywords)
        
        # If the recipe is not found, you will be informed.
        if recipes.empty:
            return f"No recipes found for the given input: {user_input}"
        
        # Random selection is made from the recipes found
        recipes = recipes.sample(min(num_options, len(recipes)))
        
        # Selected recipes are formatted
        response = "Here are some recipes that match the criteria:\n\n"
        for _, row in recipes.iterrows():
            response += self.format_recipe(row)

        print(type(response))
        
        return response

    def format_recipe(self, row: pd.Series):
        """
        Converts found recipes to a specific format

        Params:
        ---------
        row (Series): rows to be formatted

        Return:
        ---------
        formatted (str): formatted recipe
        """
        formatted = f"Recipe: {row['Name']}\n"
        formatted += f"Calories: {row['Calories']}\n"
        formatted += "Ingredients:\n" + "\n".join(f"- {ingredient}" for ingredient in row['RecipeIngredientParts'])
        formatted += "\nInstructions:\n" + "\n".join(f"{i+1}. {instruction}" for i, instruction in enumerate(row['RecipeInstructions']))
        formatted += f"\nKeywords: {', '.join(row['Keywords'])}\n\n"
        return formatted

    def chat_with_openai(self, prompt: str):
        """
        Sends the given prompt to the OpenAI model and returns the generated response.

        Params:
        ---------
        prompt (str): prompt to be sent to the model

        Return:
        ---------
        response (str): Response from the OpenAI model, with leading and trailing spaces removed
        """

        # Sends request to openAI API
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that recommends recipes based on user preferences. Analyze the given recipes and suggest the best one that matches the user's criteria. Present the chosen recipe in a clear, step-by-step format, starting with the ingredients list and followed by the preparation instructions. Do not mention that you are choosing from options or refer to option numbers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000
        )

        # the answer returns
        return response.choices[0].message.content.strip()

    def generate_recipe(self, user_input: str):
        """
        Gets recipe suggestions based on user input, creates a prompt to analyze these suggestions and selects and formats the best recipe

        Params:
        ---------
        user_input (str): Input with similar recipe search

        Return:
        ----------
        self.chat_with_openai(prompt) (str): selected formatted recipe

        """
        recipe_suggestions = self.get_recipe_suggestions(user_input)
        prompt = f"""User is looking for recipes matching the following criteria: {user_input}

        Please analyze the following recipe options and recommend the best one, explaining your choice. Then, present the chosen recipe in a clear, step-by-step format:

        1. Start with the list of ingredients.
        2. Follow with the preparation instructions, clearly numbered.
        3. If the user inputs anything such as ingredients, fruits, vegetables, etc., these should also be in the recipe. 
        4. Preparation Time: Estimated total preparation and cooking time

        Ensure that any specific ingredients or preferences mentioned in the user input are addressed in your response. Aim for a clear, concise, and informative presentation that a home cook can easily follow.

        Here are the recipe options:

        {recipe_suggestions}"""

        return self.chat_with_openai(prompt)



In [7]:
rg = RecipeGenerator(api_key="api_key", csv_file="recipe.csv")
result = rg.generate_recipe("Breakfast with high protein and bluberies calories between 300-600")
print(result)

I recommend a Blueberry Greek Yogurt Smoothie as it's a fulfilling breakfast option packed with protein from Greek yogurt and falls within the desired calorie range of 300-600 calories.

**Ingredients:**
- 1 cup Greek yogurt
- 1/2 cup blueberries
- 1 banana
- 1/4 cup oats
- 1 tablespoon honey (optional)
- 1/2 cup almond milk (adjust for desired consistency)
- Ice cubes (as needed)

**Preparation:**
1. Peel the banana and rinse the blueberries.
2. In a blender, combine Greek yogurt, blueberries, banana, oats, and honey.
3. Pour in almond milk and add ice cubes based on preference for thickness.
4. Blend all the ingredients until smooth and creamy.
5. Taste the smoothie and adjust sweetness by adding more honey if desired.
6. Pour the smoothie into a glass and enjoy your protein-packed Blueberry Greek Yogurt Smoothie!

**Preparation Time:** 10 minutes

This Blueberry Greek Yogurt Smoothie offers a delicious and protein-rich breakfast option within the desired calorie range. Enjoy the fru